Ablauf:
Fiel "AQM_import_API"
1. Import Data via API from Refinitiv to Jupiter Notebook
    - 10 Commodities 10 Years
    - Download monthly data for at least 10 asset prices and at least 10 macroeconomic variables, across countries

2. Create DataFrame, check Data and clean Data if needed:
    - N/A
    - Sprünge?
    - Format?
3. Save clean Data in SQL Lite in DB


Zuerst den Code für 1 Kombination schreiben. Danach ein Loop generieren.

File "AQM_Analyse":
1. Import the clean Data from SQL DB into Notebook
2. Prepare Data for Analysis
3. Create Dataframe to save the Results
4. Create Loop, which executes multivariate OLS regressions using all combinations of 1 and 10 explanatory macroeconomic variables 
5. Save the Results from the Loop into the "Result"-Dataframe
    For each Regression calculate:
        Autocorrelation
        heteroskedasticity
        multicollinearity
6. Export Results to SQL Lite into Table "Results"

Todo:
- Direkt von API in Dataframe und danach in SQL ohne Excel
- Excel workaround sauber fertig machen

In [56]:
import pandas as pd
import numpy as np

import scipy
from scipy import stats
import statsmodels as sm
from statsmodels.regression.linear_model import OLS

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline


from datetime import datetime
import time
import datetime

import eikon as ek
creds = 'f5a341b614d945b9b93a71fe53348ba119d11e99'
#API Key Lionel: d505f265d9bd4c0cb7df44481ece31a0ad8d1974
#API Key Raphi: f5a341b614d945b9b93a71fe53348ba119d11e99
#API Key Laurin:
ek.set_app_key(creds)


In [57]:
#Define the Future Contracts
Futures = ['LCOc1',     #ICE Europe Brent Crude 
           'CLc1',      #NYMEX Light Sweet Crude Oil (WTI)
           'NGLNMc1',   #Intercontinetanl Exchange UK NBP Natrual Gas
           'NGc1',      #NYMEX Henry Hub Natural Gas
           'GCc1',      #COMEX Gold
           'SIc1',      #COMEX Silver
           'PLc1',      #NYM Platinum
           'PAc1',      #CYM Palladium
           'Wc1',       #CBoT Wheat
           'Cc1',       #CBoT Corn
           'LCc1',      #CME Live Cattle
           'OJc1'       #ICE Orange Juice
          ]

In [58]:
data_all = pd.DataFrame()
try:
    ek.set_app_key('f5a341b614d945b9b93a71fe53348ba119d11e99')
    itemName=Futures
    dateStart="2012-01-01"
    dateEnd="2022-01-01"

    #Create Loop to inport all Futures:
    for y in range(0,len(Futures)):
        data_input = ek.get_timeseries(Futures[y],fields=["Close"], start_date = dateStart , 
                                    end_date = dateEnd , interval='monthly', corax = 'adjusted' )
        data_input['Instrument'] = Futures[y]
        data_all = data_all.append(data_input)
except ek.EikonError as ex:
    print('Eikon Error')
    print(ex)
except Exception as ex:
    print(ex)

data_all

,CLOSE,Instrument
Date,,
2012-01-31,110.98,LCOc1
2012-02-29,122.66,LCOc1
2012-03-31,122.88,LCOc1
2012-04-30,119.47,LCOc1
2012-05-31,101.87,LCOc1
...,...,...
2021-08-31,134.55,OJc1
2021-09-30,132.50,OJc1
2021-10-31,120.35,OJc1


In [59]:
data_all['Return'] = np.log(data_all.CLOSE) - np.log(data_all.CLOSE.shift(1))
data_all = data_all.reset_index()
data_all['Date'] = pd.to_datetime(data_all['Date']).dt.date
data_all.to_excel('Commodities_returns.xlsx')
liste_SQL = data_all.values.tolist()
data_all

,Date,CLOSE,Instrument,Return
0,2012-01-31,110.98,LCOc1,NaN
1,2012-02-29,122.66,LCOc1,0.100066
2,2012-03-31,122.88,LCOc1,0.001792
3,2012-04-30,119.47,LCOc1,-0.028143
4,2012-05-31,101.87,LCOc1,-0.159368
...,...,...,...,...
1435,2021-08-31,134.55,OJc1,0.011963
1436,2021-09-30,132.50,OJc1,-0.015353
1437,2021-10-31,120.35,OJc1,-0.096178
1438,2021-11-30,122.90,OJc1,0.020967


Save Data in SQL Database (DB)

In [60]:
import sqlite3
from sqlite3 import Error
 
""" create a database connection tool to a SQLite database """
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")

2.6.0


In [61]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS COMMODITIES_RETURNS;''')

In [62]:
conn = sqlite3.connect('SQL_DB.db')
c = conn.cursor()
c.execute('''CREATE TABLE COMMODITIES_RETURNS (
    DATE DATETIME,
	"CLOSE" NUMERIC,
    "Instrument" TEXT,
    "Return" NUMERIC
);''')


In [63]:
conn = sqlite3.connect(r"C:\Users\lione\OneDrive\Dokumente\GitHub\AQM\SQL_DB.db")
#Add all Data at once:
c = conn.cursor()
c.executemany("INSERT INTO COMMODITIES_RETURNS(DATE, CLOSE, Instrument, Return) VALUES (?,?,?,?)", liste_SQL)    
conn.commit()


In [66]:
#Select Data from SQL DB:   
c = conn.cursor()
c.execute("SELECT * FROM COMMODITIES_RETURNS")
new_data=c.fetchall()
ref_data = pd.DataFrame(new_data,columns=['DATE', 'CLOSE', 'Instrument', 'Return'])
ref_data = ref_data.set_index('DATE')
ref_data

,CLOSE,Instrument,Return
DATE,,,
2012-01-31,110.98,LCOc1,NaN
2012-02-29,122.66,LCOc1,0.100066
2012-03-31,122.88,LCOc1,0.001792
2012-04-30,119.47,LCOc1,-0.028143
2012-05-31,101.87,LCOc1,-0.159368
...,...,...,...
2021-08-31,134.55,OJc1,0.011963
2021-09-30,132.50,OJc1,-0.015353
2021-10-31,120.35,OJc1,-0.096178


In [67]:
Comm_returns = ref_data.pivot(columns='Instrument', values=['Return'])
Comm_returns

Return                                                    \
Instrument      CLc1       Cc1      GCc1     LCOc1      LCc1   NGLNMc1   
DATE                                                                     
2012-01-31  0.235969 -0.187460  6.143967       NaN -1.558503 -0.290478   
2012-02-29  0.083629  0.027018 -0.016185  0.100066  0.022179  0.054151   
2012-03-31 -0.038560 -0.019224 -0.024031  0.001792 -0.058057  0.035080   
2012-04-30  0.017798  0.024920 -0.003541 -0.028143 -0.014214 -0.060838   
2012-05-31 -0.192230 -0.173200 -0.062513 -0.159368 -0.013992 -0.069586   
...              ...       ...       ...       ...       ...       ...   
2021-08-31 -0.076555 -0.024053  0.001323 -0.044744 -0.025932  0.207778   
2021-09-30  0.091054  0.005137 -0.033446  0.073031  0.013569  0.676408   
2021-10-31  0.107797  0.057029  0.015658  0.071977  0.054864 -0.414303   
2021-11-30 -0.233306 -0.002202 -0.005286 -0.178725  0.064600  0.361705   
2021-12-31  0.127906  0.045257  0.029938  0.097279  0.022019 -0.334016   

                                                                        
Instrument      NGc1      OJc1      PAc1      PLc1      SIc1       Wc1  
DATE                                                                    
2012-01-31 -4.222066  0.413353 -0.340701  4.219001 -4.007161 -1.052574  
2012-02-29  0.044156 -0.097193  0.029731  0.064479  0.039819 -0.002631  
2012-03-31 -0.207404 -0.147004 -0.078729 -0.031720 -0.063076 -0.005283  
2012-04-30  0.072124 -0.155215  0.042856 -0.044050 -0.047622 -0.019871  
2012-05-31  0.058228 -0.228750 -0.107767 -0.101703 -0.109752 -0.006194  
...              ...       ...       ...       ...       ...       ...  
2021-08-31  0.111804  0.011963 -0.073216 -0.032029 -0.063390  0.004254  
2021-09-30  0.292980 -0.015353 -0.260308 -0.053689 -0.084389  0.026184  
2021-10-31 -0.078141 -0.096178  0.042952  0.059295  0.083763  0.063095  
2021-11-30 -0.172346  0.020967 -0.153723 -0.095644 -0.050535  0.001293  
2021-12-31 -0.202448  0.174288  0.113658  0.039984  0.024430 -0.003885  

[120 rows x 12 columns]